<a href="https://colab.research.google.com/github/steffiangel/KARBON/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

with open("data.json", "r") as file:
    data = json.load(file)
print("Metadata:", data.get("metadata"))


company_info = data.get("data", {}).get("company", {})
print("Company Name:", company_info.get("legal_name"))
print("Paid-up Capital:", company_info.get("paid_up_capital"))


pan = company_info.get("pan")
website = company_info.get("website")

pan_flag = FLAGS.WHITE if not pan else FLAGS.GREEN
website_flag = FLAGS.WHITE if not website else FLAGS.GREEN

print(f"PAN Flag: {pan_flag}, Website Flag: {website_flag}")

Metadata: {'api_version': '1.3', 'last_updated': '2023-01-20'}
Company Name: TEST PRIVATE LIMITED
Paid-up Capital: 1179.76
PAN Flag: 1, Website Flag: 4


In [7]:
import json
import datetime

class FLAGS:
    GREEN = 1
    AMBER = 2
    RED = 0
    MEDIUM_RISK = 3  # display purpose only
    WHITE = 4  # data is missing for this field

def latest_financial_index(data: dict) -> int:
    financials = data.get("data", {}).get("financials", [])
    latest_index = None
    latest_date = None

    for index, financial in enumerate(financials):
        if financial.get("nature") == "STANDALONE":
            date_str = financial.get("year")
            if date_str:
                try:
                    current_date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
                    print(f"Valid standalone entry at index {index}: {current_date}")
                except ValueError:
                    print(f"Invalid date format in entry {index}: {date_str}")
                    continue

                if latest_date is None or current_date > latest_date:
                    latest_date = current_date
                    latest_index = index

    return latest_index if latest_index is not None else 0

def calculate_total_revenue(data: dict, financial_index: int) -> float:
    financials = data.get("data", {}).get("financials", [])
    if financial_index < 0 or financial_index >= len(financials):
        print(f"Invalid index: {financial_index}. Returning 0.")
        return 0.0

    financial_entry = financials[financial_index]
    pnl = financial_entry.get("pnl", {})
    line_items = pnl.get("lineItems", {})
    total_revenue = line_items.get("operating_profit", 0) + line_items.get("other income", 0)
    return total_revenue

def total_borrowing(data: dict, financial_index: int) -> float:
    financials = data.get("data", {}).get("financials", [])
    if financial_index < 0 or financial_index >= len(financials):
        print(f"Invalid index: {financial_index}. Returning 0.")
        return 0.0

    financial_entry = financials[financial_index]
    bs = financial_entry.get("bs", {})
    long_term_borrowings = bs.get("long_term_borrowings", 0)
    short_term_borrowings = bs.get("short_term_borrowings", 0)

    total_borrowings = long_term_borrowings + short_term_borrowings
    revenue = calculate_total_revenue(data, financial_index)

    if revenue == 0:
        print("Total revenue is 0. Returning 0 for borrowing ratio.")
        return 0.0

    borrowing_ratio = total_borrowings / revenue
    return borrowing_ratio

def iscr(data: dict, financial_index: int) -> float:
    financials = data.get("data", {}).get("financials", [])
    if financial_index < 0 or financial_index >= len(financials):
        print(f"Invalid index: {financial_index}. Returning 0.")
        return 0.0

    financial_entry = financials[financial_index]
    pnl = financial_entry.get("pnl", {})
    line_items = pnl.get("lineItems", {})

    profit_before_interest_and_tax = line_items.get("profit before_interest_and_tax", 0)  # Adjust field names as necessary
    depreciation = line_items.get("depreciation", 0)
    interest_expense = line_items.get("interest", 0)

    is_service_coverage_ratio = (profit_before_interest_and_tax + depreciation + 1) / (interest_expense + 1)
    return is_service_coverage_ratio

def iscr_flag(data: dict, financial_index: int):
    is_service_coverage_ratio = iscr(data, financial_index)
    return FLAGS.GREEN if is_service_coverage_ratio >= 2 else FLAGS.RED

def total_revenue_5cr_flag(data: dict, financial_index: int) -> int:
    total_revenue = calculate_total_revenue(data, financial_index)
    return FLAGS.GREEN if total_revenue >= 50000000 else FLAGS.RED

def borrowing_to_revenue_flag(data: dict, financial_index: int) -> int:
    borrowing_ratio = total_borrowing(data, financial_index)
    return FLAGS.GREEN if borrowing_ratio <= 0.25 else FLAGS.AMBER

with open("data.json", "r") as file:
    data = json.load(file)

index = latest_financial_index(data)
print(f"The index of the latest standalone financial entry is: {index}")
total_revenue = calculate_total_revenue(data, index)
print(f"The total revenue from the financial data at index {index} is: {total_revenue}")
borrowing_ratio = total_borrowing(data, index)
print(f"The ratio of total borrowings to total revenue at index {index} is: {borrowing_ratio:.2f}")

# Determine flags
flag_color = iscr_flag(data, index)
flag_color1 = total_revenue_5cr_flag(data, index)
borrowing_flag_color = borrowing_to_revenue_flag(data, index)

# Print results for flags
if borrowing_flag_color == FLAGS.GREEN:
    print("The borrowing to revenue flag color is: GREEN (Ratio is less than or equal to 0.25)")
elif borrowing_flag_color == FLAGS.AMBER:
    print("The borrowing to revenue flag color is: AMBER (Ratio is greater than 0.25)")

if flag_color1 == FLAGS.GREEN:
    print("The total revenue flag color is: GREEN (Revenue exceeds 50 million)")
elif flag_color1 == FLAGS.RED:
    print("The total revenue flag color is: RED (Revenue is below 50 million)")

if flag_color == FLAGS.GREEN:
    print("The ISCR flag color is: GREEN")
elif flag_color == FLAGS.RED:
    print("The ISCR flag color is: RED")



Valid standalone entry at index 0: 2022-03-31 00:00:00
Valid standalone entry at index 1: 2021-03-31 00:00:00
Valid standalone entry at index 2: 2020-03-31 00:00:00
Valid standalone entry at index 3: 2019-03-31 00:00:00
Valid standalone entry at index 4: 2018-03-31 00:00:00
Valid standalone entry at index 5: 2017-03-31 00:00:00
Valid standalone entry at index 6: 2016-03-31 00:00:00
Valid standalone entry at index 7: 2015-03-31 00:00:00
The index of the latest standalone financial entry is: 0
The total revenue from the financial data at index 0 is: 599710
The ratio of total borrowings to total revenue at index 0 is: 0.00
The borrowing to revenue flag color is: GREEN (Ratio is less than or equal to 0.25)
The total revenue flag color is: RED (Revenue is below 50 million)
The ISCR flag color is: GREEN
